In [3]:
from O2DESPy.sandbox import Sandbox
from datetime import timedelta
import random


class Generator(Sandbox):
    def __init__(self, hourly_rate, seed=0):
        super().__init__()
        self.hourly_rate = hourly_rate
        self.count = 0
        self.on_generate = self.create_event()
        # self.on_generate = []
        
        self.schedule((self.generate, 'test'))

    def generate(self, test):
        if self.count > 0:
            print("{0}\t{1}\tGenerate. Count: {2}".format(self.clock_time, type(self).__name__, self.count))
            self.invoke(self.on_generate)
        self.count += 1
        self.schedule((self.generate, 'test'), timedelta(hours=round(random.expovariate(1 / self.hourly_rate))))
        # self.schedule([self.generate], {'test': 'test'}), timedelta(hours=round(random.expovariate(1 / self.hourly_rate))))

class Queue(Sandbox):
    def __init__(self, seed=0):
        super().__init__()
        self.seed = seed
        self.number_waiting = 0

    def enqueue(self):
        self.number_waiting += 1
        print("{0}\t{1}\tEnqueue. #Waiting: {2}".format(self.clock_time, type(self).__name__, self.number_waiting))

    def dequeue(self):
        self.number_waiting -= 1
        print("{0}\t{1}\tDequeue. #Waiting: {2}".format(self.clock_time, type(self).__name__, self.number_waiting))

class Server(Sandbox):
    def __init__(self, capacity, hourly_service_rate, seed=0):
        super().__init__()
        self.capacity = capacity
        self.hourly_service_rate = hourly_service_rate
        self.number_pending = 0
        self.number_in_service = 0
        self.on_start = self.create_event()
        self.seed = seed

    def request_to_start(self):
        self.number_pending += 1
        print("{0}\t{1}\tRequestToStart. #Pending: {2}. #In-Service: {3}".format(self.clock_time, type(self).__name__, self.number_pending, self.number_in_service))
        if self.number_in_service < self.capacity:
            self.start()
    
    def start(self):
        self.number_pending -= 1
        self.number_in_service += 1
        print("{0}\t{1}\tStart. #Pending: {2}. #In-Service: {3}".format(self.clock_time, type(self).__name__, self.number_pending, self.number_in_service))
        self.schedule(self.finish, timedelta(hours=round(random.expovariate(1 / self.hourly_service_rate))))
        self.invoke(self.on_start)

    def finish(self):
        self.number_in_service -= 1
        print("{0}\t{1}\tStart. #Pending: {2}. #In_Service: {3}".format(self.clock_time, type(self).__name__, self.number_pending, self.number_in_service))
        if self.number_pending > 0:
            self.start()
            
class MMcQueuePull(Sandbox):
    def __init__(self, capacity, hourly_arrival_rate, hourly_service_rate, seed=0):
        super().__init__()
        self.capacity = capacity
        self.hourly_arrival_rate = hourly_arrival_rate
        self.hourly_service_rate = hourly_service_rate
        self.generator = self.add_child(Generator(self.hourly_arrival_rate))
        self.queue = self.add_child(Queue())
        self.server = self.add_child(Server(self.capacity, self.hourly_service_rate))
        
        self.generator.on_generate.add_event_method(self.queue.enqueue)
#         self.generator.on_generate = [self.queue.enqueue]
        self.generator.on_generate.add_event_method(self.server.request_to_start)
        self.server.on_start.add_event_method(self.queue.dequeue)

In [6]:
import datetime

sim1 = MMcQueuePull(capacity=1, hourly_arrival_rate=5, hourly_service_rate=5)
hc1 = sim1.add_hour_counter()
sim1.run(duration=datetime.timedelta(hours=100))

0001-01-01 03:00:00	Generator	Generate. Count: 1
0001-01-01 03:00:00	Queue	Enqueue. #Waiting: 1
0001-01-01 03:00:00	Server	RequestToStart. #Pending: 1. #In-Service: 0
0001-01-01 03:00:00	Server	Start. #Pending: 0. #In-Service: 1
0001-01-01 03:00:00	Queue	Dequeue. #Waiting: 0
0001-01-01 04:00:00	Server	Start. #Pending: 0. #In_Service: 0
0001-01-01 21:00:00	Generator	Generate. Count: 2
0001-01-01 21:00:00	Queue	Enqueue. #Waiting: 1
0001-01-01 21:00:00	Server	RequestToStart. #Pending: 1. #In-Service: 0
0001-01-01 21:00:00	Server	Start. #Pending: 0. #In-Service: 1
0001-01-01 21:00:00	Queue	Dequeue. #Waiting: 0
0001-01-01 22:00:00	Generator	Generate. Count: 3
0001-01-01 22:00:00	Queue	Enqueue. #Waiting: 1
0001-01-01 22:00:00	Server	RequestToStart. #Pending: 1. #In-Service: 1
0001-01-02 00:00:00	Generator	Generate. Count: 4
0001-01-02 00:00:00	Queue	Enqueue. #Waiting: 2
0001-01-02 00:00:00	Server	RequestToStart. #Pending: 2. #In-Service: 1
0001-01-02 01:00:00	Server	Start. #Pending: 2. #In_S

True